# DFT MD on disordered `HfNbTaZr` alloy

A disordered equiatomic `HfNbTaZr` crystal was generated in Vesta using atomic positions seeded randomly using `mcsqs`.

This notebook handles a few things:
1. Slab relaxation, if `RUN_RELAXATION=True`
2. MD run at configurable # time steps, dt, initial eV
3. Previewing system (`preview(...)`)

Use `ase gui <output_filename.out>` to view animated relax/MD output.

# Helpers

In [1]:
from ase import Atom, Atoms
from ase.calculators.espresso import Espresso
from ase.constraints import FixAtoms
from ase.io.cif import read_cif
from ase.io.espresso import read_espresso_out, write_espresso_in
from ase.io.vasp import read_vasp
from ase.visualize import view
from ase.visualize.plot import plot_atoms

from copy import deepcopy
from matplotlib import pyplot as plt
import math
import numpy as np
from datetime import datetime

from qe_utils import (velocity,
    import_vasp,
    output_to_atoms,
    relax,
    pin_bottom_layers,
    get_D_position,
    preview,
    md,
    sanitize)

In [2]:
# Numeric constants
# 1 picosecond = n Rydberg a.u.
PS_TO_AU = 1e-12 / (4.8378 * 1e-17)

# 1 femtosecond
FS_TO_AU = 1e-15 / (4.8378 * 1e-17)

In [3]:
#########################################################################
# RUN_RELAXATION
#    - True: run a relaxation step (can take several hours)
#    - False: don't run a relaxation step, use an existing relaxed crystal
# An existing relaxed crystal exists in relax_data/relax_Hf5Nb2Ta10Zr5.out
#########################################################################
RUN_RELAXATION = False

#########################################################################
# VACUUM
#    - Amount of vacuum, in Angstroms, to place on either side of the system
#########################################################################
VACUUM = 2.0

#########################################################################
# DEUTERIUM_MASS_AMU
#    - Deuterium mass in AMU
#########################################################################
DEUTERIUM_MASS_AMU = 2.014

#########################################################################
# INITIAL_DISTANCE_A
#    - Initial distance between D atom and surface of slab, in Angstroms
#########################################################################
INITIAL_DISTANCE_A = 2.0

#########################################################################
# AXIS
#    - One of { 'x', 'y', 'z' }: axis along which the D atom should travel
#########################################################################
AXIS = 'x'

#########################################################################
# N_STEPS
#    - Number of steps to run MD for
#########################################################################
N_STEPS = 20

#########################################################################
# INITIAL_EV
#    - Initial eV to impart on D atom
#########################################################################
#INITIAL_EV = 1000

#########################################################################
# DT
#    - dt, in AU
#########################################################################
DT = 0.2 * round(FS_TO_AU) # 0.2fs

#INCIDENT_ANGLE_DEG = 45
#POLAR_ANGLE_DEG = 45

## Add vacuum and a single `D` atom

Fix the crystal in place & add vacuum on either side. Then add an unconstrained `D` atom `INITIAL_DISTANCE_A` Angstroms away from the slab.

In [4]:
def setup(incident_angle_deg, polar_angle_deg):
    # Run relaxation, if needed
    if RUN_RELAXATION:
        slab = import_vasp('input/HfNbTaZr_8.vasp', truncate=False)
        slab.center(vacuum=1)
        relax_output_filename = relax(slab)

    # Create our slab
    slab = output_to_atoms(relax_output_filename if RUN_RELAXATION else 'relax_data/relax_Hf5Nb2Ta10Zr5.out') # This slab is the result of relaxing a 22-atom crystal
    atoms = deepcopy(slab)
    atoms.center(vacuum=VACUUM, axis=2)
    atoms = pin_bottom_layers(atoms, nlayers=1, axis=AXIS)
    
    # Place the D atom in the center of the slab, `INITIAL_DISTANCE_A` Angstroms away
    DEUTERIUM_XYZ = get_D_position(atoms, initial_distance_a=INITIAL_DISTANCE_A, axis=AXIS, normal_angle_deg=incident_angle_deg, polar_angle_deg=polar_angle_deg)
    deuterium = Atom('H', mass=DEUTERIUM_MASS_AMU, position=DEUTERIUM_XYZ)
    atoms.append(deuterium)

    # Expand unit cell so that the D atom fits
    existing_cell = atoms.get_cell()
    atoms.set_cell(np.array([ # TODO this code is dogshit, clean it up
        existing_cell[0][0] + (2 * INITIAL_DISTANCE_A if AXIS == 'x' else 0),
        existing_cell[1][1] + (2 * INITIAL_DISTANCE_A if AXIS == 'y' else 0),
        existing_cell[2][2] + (2 * INITIAL_DISTANCE_A if AXIS == 'z' else 0)]
    ))

    return atoms
    #preview(atoms)

## Run MD

In [ ]:
counter = 0
for INCIDENT_ANGLE_DEG in [0, 45]:
    for POLAR_ANGLE_DEG in range(0, 181, 30):
        for INITIAL_EV in [50, 100]:
            atoms = setup(INCIDENT_ANGLE_DEG, POLAR_ANGLE_DEG)
            print(f'{counter+1}: Running MD for incident angle={INCIDENT_ANGLE_DEG}deg, polar angle={POLAR_ANGLE_DEG}deg, D eV={INITIAL_EV}eV. {N_STEPS} steps, {DT} integration timestep, starting {INITIAL_DISTANCE_A}angstrom away')
            output_filename = md(
                atoms,
                nsteps=N_STEPS,
                dt=DT,
                initial_eV=INITIAL_EV,
                incident_angle_deg=INCIDENT_ANGLE_DEG,
                polar_angle_deg=POLAR_ANGLE_DEG,
                is_cluster=False,
                ncores=12
            )
            sanitize(output_filename)
            print('-------------------------------------------------------')

1: Running MD for incident angle=0deg, polar angle=0deg, D eV=50eV. 20 steps, 4.2 integration timestep, starting 2.0angstrom away
Writing D initial velocity 50eV (vx=-0.03163843088698983, vy=0.0, vz=0.0 Hartree au)
Running command: mpirun -np 12 q-e/bin/pw.x -inp md_data/md_Hf5Nb2Ta10Zr5H_20steps_50eV_incident0_polar0.in


From here, run `ase gui <output_filename>` to view the animation